# Libraries

In [ ]:
# import os
import json
import pandas as pd
from openai import OpenAI
from datasets import load_dataset
from sklearn.metrics import f1_score
from pprint import pprint

pd.set_option('max_colwidth', None)

# Config

In [ ]:
seed = 42

client = OpenAI(api_key=json.load(open('secrets.json'))['OPENAPI_SECRET_KEY'])

idn2eng_emotion_map = {
    'marah': 'anger',
    'jijik': 'disgust',
    'takut': 'fear',
    'senang': 'joy',
    'sedih': 'sadness',
    'terkejut': 'surprise',
    'biasa': 'neutral',
}

# Data

## Load Data

In [ ]:
df = pd.read_csv('data/preprocessed_data/track_a/sun_70_15_15_stratify_v2/val.csv')
df = df.rename(columns=idn2eng_emotion_map)
emotion_cols = [col for col in df.columns if col not in ['Unnamed: 0', 'text', 'emotion', 'id']]
df['emotion'] = df.apply(lambda row: ', '.join([emotion for emotion in emotion_cols if row[emotion] == 1]), axis=1)

print("DF size:", len(df))
print("Emotion columns:", emotion_cols)
df.head()

In [ ]:
# Omit data with 'emotion' "neutral"
# df = df[~(df['emotion'] == 'neutral')]
# print("DF size (after removing emotion 'neutral'):", len(df))

# Evaluation

In [223]:
# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, meaning the identified emotion(s) may include one or more labels from the predefined EMOTION LABELS below.

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# The OUTPUTS must consist of KEYWORD(S) and EMOTION(S).
# Only respond the KEYWORD(S) and EMOTION(S).

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) must at least include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) must at least include "joy."

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.
# - The emoji 😭 does not always indicate "sadness". It can sometimes represent extreme laughter that brings tears. In such cases, the identified emotion(s) should include "joy."

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.
# - The emoji 😭 does not always indicate "sadness". It can sometimes represent extreme laughter that brings tears. In such cases, the identified emotion(s) should include "joy."

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# F1 Macro: 0.5238851095993953
prompt = """# INSTRUCTION
Identify the EMOTION(S) expressed in the TEXT. 
This is a multi-label classification task, meaning there will always be one or more emotions to identify in the TEXT. 
The model must output the EMOTION(S) from the predefined EMOTION LABELS listed below. 
The model must always return a result—there should never be an instance with no identified EMOTION(s).

## EMOTION LABELS
```
- anger
- fear
- joy
- sadness
- surprise
```

## STEPS TO IDENTIFY THE EMOTION(S)
1. Guess the CONTEXT of the TEXT by analyzing its overall meaning and tone. Provide a concise description of the CONTEXT to clarify the emotional environment in which the TEXT is situated.
2. Using the guessed CONTEXT, Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`. Ensure that the extracted KEYWORD(S) align with the described CONTEXT to maintain consistency in interpretation.
3. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

## TIPS
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate `joy`.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate `joy`.
- Insults typically indicate `anger` and may also suggest `disgust`.
- The emoji 😭 does not always indicate `sadness`. In humorous contexts, it can represent extreme laughter that brings tears, indicating `joy`.
- In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.

The OUTPUTS must include the CONTEXT, KEYWORD(S) and EMOTION(S).
Do not omit any emotion expressed in the TEXT.

# INPUT
## TEXT
{text}

# OUTPUTS
## CONTEXT
"""

prompt_1_1 = """You will be provided with a text in Sundanese. Your task is to identify the emotion(s) expressed in the text. This is a multi-label classification task, meaning the text may convey one or more emotions simultaneously. You must always provide a result; there should never be an instance with no identified emotion(s).

## PREDEFINED EMOTION LABELS
```
- anger
- fear
- joy
- sadness
- surprise
```

## STEP-BY-STEP INSTRUCTIONS
1. Extract all keyword(s) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `emotion` using the predefined labels in the format: `keyword -> emotion (reason)`.
2. Based on the identified keyword(s), determine the overall emotion(s) expressed in the TEXT.

## ADDITIONAL CONTEXT
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate `joy`.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate `joy`.
- Insults typically indicate `anger` and may also suggest `disgust`.
- The emoji 😭 does not always indicate `sadness`. In humorous contexts, it can represent extreme laughter that brings tears, indicating `joy`.
- In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.

## OUTPUT FORMAT
```
<keywords>
A markdown list of one or more keywords, each in the format: `keyword -> emotion (reason)`.
</keywords>

<emotions>
A list of one or more emotions separated by commas.
</emotions>
```

Remember: Do not omit any emotion expressed in the text.

"""

print(prompt_1_1)

You will be provided with a text in Sundanese. Your task is to identify the emotion(s) expressed in the text. This is a multi-label classification task, meaning the text may convey one or more emotions simultaneously. You must always provide a result; there should never be an instance with no identified emotion(s).

## PREDEFINED EMOTION LABELS
```
- anger
- fear
- joy
- sadness
- surprise
```

## STEP-BY-STEP INSTRUCTIONS
1. Extract all keyword(s) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `emotion` using the predefined labels in the format: `keyword -> emotion (reason)`.
2. Based on the identified keyword(s), determine the overall emotion(s) expressed in the TEXT.

## ADDITIONAL CONTEXT
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate `joy`.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate `joy`.
- Insults typically indicate `anger` and may also suggest `dis

In [231]:
def predict(text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                'content': prompt.format(text=text),
            },
        ],
        max_tokens=300,
    )
    return completion.choices[0].message.content

def predict_1_1(text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt_1_1},
            {"role": "user", 'content': text},
        ],
        max_tokens=300,
    )
    return completion.choices[0].message.content

def one_hot_encode_emotions(emotions, emotion_cols):
    one_hot_emotion = [1 if emotion_col in emotions else 0 for emotion_col in emotion_cols]
    return one_hot_emotion

print(predict("Cek urang mah jelema goblog teh nyaeta, jelema nu menta udud ngomong teu boga, padahal Aya sabungkus dinu pesak 😂🤣"))
print()
print(predict_1_1("Cek urang mah jelema goblog teh nyaeta, jelema nu menta udud ngomong teu boga, padahal Aya sabungkus dinu pesak 😂🤣"))

The TEXT describes a situation where someone is humorously pointing out the hypocrisy or silliness in a person's behavior when they claim not to have a cigarette while they actually have a full pack. This sarcasm is highlighted by the laughter emojis at the end.

## KEYWORD(S)
- "goblog" -> anger (insulting term indicating frustration or anger towards the perceived stupidity)
- "menta udud ngomong teu boga, padahal Aya sabungkus dinu pesak" -> surprise (expressing disbelief at the contradictory statement)
- "😂🤣" -> joy (laughter emojis indicate amusement at the situation)

## EMOTION(S)
anger, surprise, joy

<keywords>
- goblog -> anger (an insult directed towards someone, indicating anger)
- 😂 -> joy (laughter emoji indicating humor)
- 🤣 -> joy (laughter emoji indicating humor)
</keywords>

<emotions>
anger, joy
</emotions>


In [ ]:
# df_sampled = df.sample(n=20)
# y_true = df_sampled[emotion_cols].to_numpy()

# y_pred = []
# for i, v in enumerate(df_sampled.iterrows()):
#     idx, row = v

#     print("=" * 64)
#     print(f"{i+1} of {len(df_sampled)}")
#     print("=" * 64)

#     is_valid = True

#     while True:
#         print("Predicting...")        

#         output = predict(row['text'])
#         ctx = prompt + output

#         keywords_emotions = ctx.split("# KEYWORD(S)")[-1].split("## EMOTION(S)")
#         # keywords = keywords_emotions[0].strip().split("\n")
#         keywords = keywords_emotions[0].strip()
#         emotions = [emotion.replace("-", "").strip() for emotion in keywords_emotions[-1].strip().split("\n")]

#         is_valid = bool(sum([int(emotion_pred in emotion_cols) for emotion_pred in emotions]))

#         if is_valid:
#             break
#         else:
#             print("ERROR:", output)
#             print()
        
#     emotions_one_hot = one_hot_encode_emotions(emotions, emotion_cols)
#     y_pred.append(emotions_one_hot)

#     print()
#     print("# TEXT")
#     print(row['text'])
#     print()
#     print("# KEYWORD(S)")
#     print(keywords)
#     print()
#     print("# PREDICTED EMOTION(S)")
#     print(emotions)
#     print()
#     print("# TRUE EMOTION(S)")
#     print(row['emotion'])
#     print()
#     # df.at[i, 'text_sun2idn'] = text_sun2idn

# f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=1.0)
# print("F1 Macro:", f1_macro)

In [ ]:
# # df_sampled = df.sample(n=20)
# y_true = df[emotion_cols].to_numpy()

# y_pred = []
# for i, v in enumerate(df.iterrows()):
#     idx, row = v

#     print("=" * 64)
#     print(f"{i+1} of {len(df)}")
#     print("=" * 64)

#     is_valid = True

#     while True:
#         print("Predicting...")        

#         output = predict(row['text'])
#         prompt_output = prompt + output

#         outputs = prompt_output.split("# OUTPUTS")[-1]
#         ctx = outputs.split("## CONTEXT")[-1].split("## KEYWORD(S)")[0].strip()
#         keywords = outputs.split("## KEYWORD(S)")[-1].split("## EMOTION(S)")[0].strip()
#         emotions = outputs.split("## EMOTION(S)")[-1].strip()
#         emotions = [emotion.replace("-", "").strip() for emotion in emotions.split("\n")]

#         is_valid = bool(sum([int(emotion_pred in emotion_cols) for emotion_pred in emotions]))

#         if is_valid:
#             break
#         else:
#             print("ERROR:", output)
#             print()
        
#     emotions_one_hot = one_hot_encode_emotions(emotions, emotion_cols)
#     y_pred.append(emotions_one_hot)

#     print()
#     print("# TEXT")
#     print(row['text'])
#     print()
#     print("# CONTEXT")
#     print(ctx)
#     print()
#     print("# KEYWORD(S)")
#     print(keywords)
#     print()
#     print("# PREDICTED EMOTION(S)")
#     print(emotions)
#     print()
#     print("# TRUE EMOTION(S)")
#     print(row['emotion'])
#     print()
#     # df.at[i, 'text_sun2idn'] = text_sun2idn

# f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=1.0)
# print("F1 Macro:", f1_macro)

## Evaluation - Prompt #2

In [202]:
# Prompt Parts
"""
1. Translate the Text
Begin by translating the Sundanese text into English. Ensure that the meaning and nuances are accurately captured during the translation process.

### Opposite Perspectives
- Consider the opposite perspectives and explore alternative emotions that could also apply. For example, if your reasoning points to emotions such as sadness, fear, or anger, evaluate whether emotions like joy or surprise might also be relevant.
- Check whether these alternative emotions make sense in the context of the text, ensuring a well-rounded and thoughtful analysis.

### Tone and Nuance
- Be aware of any exaggeration, sarcasm, or irony that may alter the emotional understanding of the text. These elements can drastically change how the emotion is interpreted.

if there's exaggeration in  ...
"""

prompt2 = """
You will be provided with a text in Sundanese. Your task is to identify the emotion(s) expressed in the text. This is a multi-label classification task, meaning the text may convey one or more emotions simultaneously. 

## Predefined Emotion Labels
- anger
- fear
- joy
- sadness
- surprise

## Step-by-Step Instructions

1. Translate the Text
Begin by translating the Sundanese text into English. Ensure that the meaning and nuances are accurately captured during the translation process.

2. Conduct In-Depth Reasoning:
Engage in extensive reasoning (minimum 10,000 characters) in a natural, conversational style, akin to capturing an internal monologue. Your analysis should aim to thoroughly understand the text's emotional content. Follow these guiding principles:

### Understanding
- Analyze the text in both Sundanese and English. Ensure your interpretations in each language align and do not contradict each other.
- If contradictions arise, revisit and refine your analysis until the logic reconciles.
- Examine every part of the text meticulously, from start to finish and in reverse order, ensuring no segment is overlooked.

### Exploration
- Adopt a mindset of self-doubt and curiosity. Ask numerous questions and explore different possibilities.
- Avoid rushing to conclusions; instead, iterate and refine your thoughts through multiple rounds of questioning.
- Include key questions such as:
    - "What event triggered the writer's emotions and motivated them to write the text?"
    - "What was the situation of the writer when this event occurred?"
    - "To whom was the text addressed, and how were the emotions conveyed?"
- Build your understanding progressively by answering questions step by step. If uncertainties remain, continue exploring, reasoning, and revising until a comprehensive understanding emerges.

3. Identify Emotions
Based on your detailed reasoning, identify the overall emotion(s) expressed in the text from the predefined labels.

## Internal Monologue Conversational Style

1. Natural Thought Flow
- "Hmm... let me think about this..."
- "Wait, that doesn't seem right..."
- "Maybe I should approach this differently..."
- "Going back to what I thought earlier..."

2. Progressive Building
- "Starting with the basics..."
- "Building on that last point..."
- "This connects to what I noticed earlier..."
- "Let me break this down further..."

## Additional Context
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate joy.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate joy.
- Insults typically indicate anger and may also suggest disgust.
- The emoji 😭 does not always indicate sadness. In humorous contexts, it can represent extreme laughter that brings tears, indicating joy.
- In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.

## Output Format

<english_translation>
English translations of the Sundanese text.
</english_translation>

<reasoning>
Your detailed reasoning.
</reasoning>

<emotions>
List of one or more identified emotions separated by commas.
</emotions>
"""

print(prompt2)


You will be provided with a text in Sundanese. Your task is to identify the emotion(s) expressed in the text. This is a multi-label classification task, meaning the text may convey one or more emotions simultaneously. 

## Predefined Emotion Labels
- anger
- fear
- joy
- sadness
- surprise

## Step-by-Step Instructions

1. Translate the Text
Begin by translating the Sundanese text into English. Ensure that the meaning and nuances are accurately captured during the translation process.

2. Conduct In-Depth Reasoning:
Engage in extensive reasoning (minimum 10,000 characters) in a natural, conversational style, akin to capturing an internal monologue. Your analysis should aim to thoroughly understand the text's emotional content. Follow these guiding principles:

### Understanding
- Analyze the text in both Sundanese and English. Ensure your interpretations in each language align and do not contradict each other.
- If contradictions arise, revisit and refine your analysis until the logic

In [200]:
def predict(text):
    completion = client.chat.completions.create(
        model='gpt-4o',
        # model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': prompt},
            {'role': 'user', 'content': text},
        ],
        max_tokens=1000,
    )
    return completion.choices[0].message.content

output = predict("Cek urang mah jelema goblog teh nyaeta, jelema nu menta udud ngomong teu boga, padahal Aya sabungkus dinu pesak 😂🤣")
print(output)

<english_translation>
In my opinion, a stupid person is someone who asks for a cigarette and says they don't have any, even though there's a packet in their pockets 😂🤣
</english_translation>

<reasoning>
Alright, let's dive into this text and see what emotions are surfacing. To start off, the English translation tells us that the author is sharing their thoughts on a person they consider "stupid." It's about someone asking for a cigarette while supposedly having a packet in their own pocket. 

Firstly, when reading this, there's a clear indication of the emotion of ridicule or irony. The word "goblog," which translates to "stupid," is a strong choice of words and suggests some frustration or annoyance with the person's behavior. It's almost like the author is pointing out a contradiction in someone's actions and labeling it as foolish. This could hint at a mild form of anger, but let's not conclude anything just yet.

What's interesting is the presence of the laughter emojis 😂🤣 at the 

## Save Evaluation

In [ ]:
save_df = df.copy()
save_df[emotion_cols] = y_pred
save_df.to_csv('eval_sun.csv', index=False)
print("Saved to: eval_sun.csv")